# Text Summarization

You task is to prepare a model that is suitable for text summarization.
We gonna use CNN_dailymail dataset

In [3]:
!pip install datasets transformers evaluate rouge_score
from datasets import load_dataset
from evaluate import load
import sentencepiece as spm

import math
import torch
from torch import nn

dataset = load_dataset("abisee/cnn_dailymail", '3.0.0')
metric = load("rouge")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3603fd9f4d402bceaaa0847f906a86dd7b71fcfb669f7aace609c711a9cdd960
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


2024-06-18 10:51:17.479247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 10:51:17.479377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 10:51:17.587965: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [22]:
all_texts = dataset['train']['article'] + dataset['validation']['article'] + dataset['test']['article']
with open('text.txt', 'w', encoding="utf-8") as ptr:
    for text in all_texts:
        ptr.write(text + '\n')

In [2]:
path = '/kaggle/working/text.txt'

# let us train sentencepiece tokenizer
spm.SentencePieceTrainer.train(
    input=path,
    model_prefix='cnn_dailymail',
    vocab_size=32000,
    model_type='bpe'
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: /kaggle/working/text.txt
  input_format: 
  model_prefix: cnn_dailymail
  model_type: BPE
  vocab_size: 32000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_priv

OSError: Not found: "/kaggle/working/text.txt": No such file or directory Error #2

In [4]:
model_path = '/kaggle/input/previous/cnn_dailymail.model'

In [5]:
# Загрузка модели
sp = spm.SentencePieceProcessor(model_file=model_path)
example_text = "Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune."
# Токенизация
tokens = sp.encode(example_text, out_type=str)
print('Original text: \n' + example_text)
print('\nText after tokenization: ')
print(tokens)
decoded_text = sp.decode(tokens)
print('\nText after detokenization:')
print(decoded_text)

Original text: 
Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune.

Text after tokenization: 
['▁Harry', '▁Potter', '▁star', '▁Daniel', '▁Radcliffe', '▁gains', '▁access', '▁to', '▁a', '▁reported', '▁£20', '▁million', '▁fortune', '.']

Text after detokenization:
Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune.


In [6]:
from transformers import T5Tokenizer

# Загрузка токенизатора
tokenizer = T5Tokenizer.from_pretrained(model_path, model_max_length=512)

# Пример текста
example_text = "Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune."

# Токенизация
inputs = tokenizer(example_text, return_tensors="pt", max_length=512, truncation=True)
print("Tokenized input ids:", inputs["input_ids"])

# Пример детокенизации
decoded_text = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
print("Decoded text:", decoded_text)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2004: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tokenized input ids: tensor([[ 3781, 10741,   648,  3345, 23291, 16234,  2520,    27,     5,   814,
          7940,   963,  9130, 31944,     2]])
Decoded text: Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune.


In [7]:
vocab_size = tokenizer.vocab_size
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 32000


In [8]:
display(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [9]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["highlights"], max_length=max_target_length, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=16)

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=16):   0%|          | 0/287113 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=16):   0%|          | 0/13368 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/11490 [00:00<?, ? examples/s]

In [11]:
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
display(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

## Now you have to prepare your dataset
Use any tokenization methods you'd like <br>
If you are using guides please cite your sourse

## Declare  your model (6 pts)
Use encoder-decoder transformer architecure of any size <br>
You can lookup for a T5 architecure





In [102]:
### CONFIG ###
from torch.nn import functional as F
import numpy as np
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
block_input = 1024
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
embedding_size = 300
n_head = 8
n_layer = 3
dropout = 0.2
vocab_size = 32101

### ATTENTION ###
class Attention(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_size, head_size, bias=False)
        self.query = nn.Linear(embedding_size, head_size, bias=False)
        self.value = nn.Linear(embedding_size, head_size, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = (q @ k.transpose(-2,-1)) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

### MultiHeadAttention ###
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Attention(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, embedding_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

    
### CrossAttention ###
class CrossAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__() 
        self.key = nn.Linear(embedding_size, head_size, bias=False)
        self.query = nn.Linear(embedding_size, head_size, bias=False)
        self.value = nn.Linear(embedding_size, head_size, bias=False)
        self.dropout = nn.Dropout(dropout) 
    
    def forward(self, x, encoder_embs):
        # we need to get encoder embeddings to get K and V matricies
        B,T,C = x.shape
        q = self.query(x) # (B,T,hs)
        k = self.key(encoder_embs) # (B,T,hs)
        v = self.value(encoder_embs) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out
    
### MultiHeadCrossAttention ###
class CrossMultiHeadAttention(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([CrossAttention(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, embedding_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_embs):
        out = torch.cat([h(x, encoder_embs) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### MaskedAttention ###
class MaskedAttention(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__() 
        self.key = nn.Linear(embedding_size, head_size, bias=False)
        self.query = nn.Linear(embedding_size, head_size, bias=False)
        self.value = nn.Linear(embedding_size, head_size, bias=False)
        self.tril = torch.tril(torch.ones(block_size, block_size)).to(device)
        #self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MaskedMultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, head_size):
        super().__init__()
        self.heads = nn.ModuleList([MaskedAttention(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(head_size * n_head, embedding_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

### LayerNorm ###

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features)).to(device)
        self.b_2 = nn.Parameter(torch.zeros(features)).to(device)
        self.eps = eps 

    def forward(self, x):
        print(self.eps.device)
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * ((x - mean) / (std + eps)) + self.b_2

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_size, 4 * embedding_size),
            nn.ReLU(),
            nn.Linear(4 * embedding_size, embedding_size),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

### EncoderBlock ###

class EncoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = embedding_size // n_head
        self.self_att = MultiHeadAttention(head_size)
        self.ffwd = FeedFoward()
        self.ln1 = nn.LayerNorm(embedding_size)
        self.ln2 = nn.LayerNorm(embedding_size)

    def forward(self, x):
        x = x + self.self_att(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

### DecoderBlock ###
class DecoderBlock(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = embedding_size // n_head
        self.masked_att = MaskedMultiHeadAttention(head_size)
        self.cross_att = CrossMultiHeadAttention(head_size)
        self.ffwd = FeedFoward()
        self.ln1 = nn.LayerNorm(embedding_size)
        self.ln2 = nn.LayerNorm(embedding_size)
        self.ln3 = nn.LayerNorm(embedding_size)
    def forward(self, x, encoder_emb):
        x = x + self.masked_att(self.ln1(x))
        x = x + self.cross_att(self.ln2(x), encoder_emb)
        x = x + self.ffwd(self.ln3(x))
        return x

### MyModel  ###
class ChatGPT5(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.token_embedding_input = nn.Embedding(vocab_size, embedding_size).to(device)
        self.position_embedding_input = nn.Embedding(block_input, embedding_size).to(device)
        
        self.token_embedding_output = nn.Embedding(vocab_size, embedding_size).to(device)
        self.position_embedding_output = nn.Embedding(block_size, embedding_size).to(device)
        
        self.encoder_blocks = nn.Sequential(*[EncoderBlock().to(device) for _ in range(n_layer)])
        
        self.decoder_blocks = []
        for i in range(n_layer):
            self.decoder_blocks.append(DecoderBlock().to(device))
         
        self.lm_head = nn.Linear(embedding_size, vocab_size).to(device)
        
    def forward(self, input_seq, target_seq):
        input_seq = input_seq.to(torch.long)
        target_seq = target_seq.to(torch.long)
        
        input_emb = self.token_embedding_input(input_seq)
        input_positions = self.position_embedding_input(torch.arange(0, block_input, dtype=torch.long, device=device))
        
        input_final = input_emb + input_positions
        
        target_emb = self.token_embedding_output(target_seq)
        target_positions = self.position_embedding_output(torch.arange(0, block_size, dtype=torch.long, device=device))
        
        target_final = target_emb + target_positions
        
        encoder_output = self.encoder_blocks(input_final)
        
        temp_input = target_final
        for i in range(n_layer):
            temp_input = self.decoder_blocks[i](temp_input, encoder_output)
        
        decoder_output = temp_input
        
        out = self.lm_head(decoder_output)
        return out
        
    def generate(self, input_sequence, tokenizer, need_to_tokenize=False, max_out_length=100):
        input_sequence = (input_sequence.to(torch.long))
        start_token, end_token = tokenizer.pad_token_id, tokenizer.eos_token_id  
        if need_to_tokenize:
            input_sequence = torch.Tensor(tokenizer.encode(input_sequence, max_length=256, padding="max_length", return_tensors="pt")).to(device) # Tokenizing string input
            input_sequence = input_sequence.to(torch.long)
        
        input_positions = self.position_embedding_input(torch.arange(0, block_input, dtype=torch.long, device=device))
        
        if len(input_sequence.shape) < 3:
            input_embeddings = self.token_embedding_input(input_sequence).unsqueeze(0)
        else:
            input_embeddings = self.token_embedding_input(input_sequence)
        final_embeddings = input_positions + input_embeddings
        encoder_output = self.encoder_blocks(final_embeddings)

        generated_tokens = []
        current_token = torch.tensor([[start_token]]).to(device)

        for _ in range(max_out_length):
            target_embeddings = self.token_embedding_output(current_token)
            
            temp_input = target_embeddings
            for i in range(n_layer):
                temp_input = self.decoder_blocks[i](temp_input, encoder_output)
            decoder_output = temp_input 
            
            output = self.lm_head(decoder_output)

            next_token = torch.argmax(output, dim=-1)

            generated_tokens.append(next_token.item())
            current_token = next_token

            if next_token.item() == end_token:
                break

        return generated_tokens

### Easy way
If you are struggling with some modules, you can use PyTorch layers:
```
nn.TransformerEncoderLayer   - 0.2 pts penalty
nn.TransformerDecoderLayer   - 0.2 pts penalty
nn.TransformerEncoder        - 0.5 pts penalty
nn.TransformerDecoder        - 0.5 pts penalty
nn.Transformer               - 1.0 pts penalty   

```



## Train your model (2 pts)

You are free to use any library, including the HuggingFace🤗

In [103]:
model = ChatGPT5()
model = model.to(device)
model

ChatGPT5(
  (token_embedding_input): Embedding(32101, 300)
  (position_embedding_input): Embedding(1024, 300)
  (token_embedding_output): Embedding(32101, 300)
  (position_embedding_output): Embedding(128, 300)
  (encoder_blocks): Sequential(
    (0): EncoderBlock(
      (self_att): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Attention(
            (key): Linear(in_features=300, out_features=37, bias=False)
            (query): Linear(in_features=300, out_features=37, bias=False)
            (value): Linear(in_features=300, out_features=37, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=296, out_features=300, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=300, out_features=1200, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1200, out_features=300, bias=True)
 

In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=batch_size, shuffle=False)

In [15]:
def train_model(model, train_dataloader, val_dataloader,
                learning_rate, n_epochs, lr_decay_factor, verbose=True,
                loss_fn=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id),
                device='cuda' if torch.cuda.is_available() else 'cpu'):
#     model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    current_lr = learning_rate
    train_epoch_losses = []
    val_epoch_losses = []

    print("Training...")
    for epoch in range(1, n_epochs + 1):
        current_lr *= lr_decay_factor
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_lr

        # Training step
        model.train()
        train_epoch_loss = 0
        for train_batch_idx, batch in enumerate(train_dataloader, start=1):
            train_inputs = batch['input_ids'].to(device)
            train_targets = batch['labels'].to(device)

            if verbose: print(f"\rTrain batch: {train_batch_idx}/{len(train_dataloader)}, Avg batch loss: {train_epoch_loss/train_batch_idx:.6f}", end='')

            train_preds = model(train_inputs, train_targets)

            train_targets_shifted = train_targets[:, 1:].contiguous().view(-1)
            train_preds = train_preds[:, :-1, :].contiguous().view(-1, train_preds.size(-1))

            train_batch_loss = loss_fn(train_preds, train_targets_shifted)
            train_epoch_loss += train_batch_loss.item()

            optimizer.zero_grad()
            train_batch_loss.backward()
            optimizer.step()

        train_epoch_losses.append(train_epoch_loss/len(train_dataloader))

        # Validation step
        model.eval()
        val_epoch_loss = 0
        with torch.no_grad():
            for val_batch_idx, batch in enumerate(val_dataloader, start=1):
                val_inputs = batch['input_ids'].to(device)
                val_targets = batch['labels'].to(device)

                val_preds = model(val_inputs, val_targets)

                val_targets_shifted = val_targets[:, 1:].contiguous().view(-1)
                val_preds = val_preds[:, :-1, :].contiguous().view(-1, val_preds.size(-1))

                val_batch_loss = loss_fn(val_preds, val_targets_shifted)
                val_epoch_loss += val_batch_loss.item()

        val_epoch_losses.append(val_epoch_loss/len(val_dataloader))

        if verbose: print(f"\nEpoch: {epoch}, Train loss: {train_epoch_losses[-1]:.6f}, Val loss: {val_epoch_losses[-1]:.6f}, lr {current_lr:.6f}\n")

    print("Training complete.")
    return train_epoch_losses, val_epoch_losses

In [ ]:
train_losses, val_losses = train_model(model, train_dataloader, val_dataloader, 1e-3, 2, 0.75)

Training...
Train batch: 17945/17945, Avg batch loss: 5.737606
Epoch: 1, Train loss: 5.737900, Val loss: 5.239655, lr 0.000750

Train batch: 15983/17945, Avg batch loss: 5.040648

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Train batch: 17945/17945, Avg batch loss: 5.035937

In [106]:
from IPython.display import FileLink
torch.save(model.state_dict(), 'gpt.pth')
FileLink(r'gpt.pth') 

/kaggle/working/gpt.pth

In [98]:
model.load_state_dict(torch.load('/kaggle/input/new-version/gpt_last.pth'))

<All keys matched successfully>

In [39]:
import gc
def all_clean(model=None, train_dataloader=None, val_dataloader=None):
    if model is not None:
        del model
    if train_dataloader is not None:
        del train_dataloader
    if val_dataloader is not None:
        del val_dataloader
    gc.collect()
    torch.cuda.empty_cache()
    print("All clean")

In [40]:
all_clean()

All clean


In [101]:
for train_batch_idx, batch in enumerate(train_dataloader, start=1):
    train_inputs = batch['input_ids'].to(device)
    train_targets = batch['labels'].to(device)
    print(f'Sample text:')
    print(tokenizer.decode(train_inputs[0], skip_special_tokens=True))
    print()
    print(f'Expected:')
    print(tokenizer.decode(train_targets[0], skip_special_tokens=True))
    model.eval()
    with torch.no_grad():
        x = model.generate(train_inputs[0], tokenizer)
        print()
        print('Model answer:')
        print(tokenizer.decode(x, skip_special_tokens=True))
    break 

Sample text:
By. Associated Press. Stacey Dean Rambold was only given a one month prison sentence for his rape against a 14-year-old victim he said 'looked older than her age' A former Billings, Montana teacher convicted of rape wrongly blamed his 14-year-old victim when he argued that she bore responsibility in the case, an attorney for the state said on Thursday in a court filing. The office of Attorney General Tim Fox is appealing Stacey Dean Rambold's one-month prison sentence to the Montana Supreme Court. The state says he should have served a mandatory minimum of four years after pleading guilty to sexual assault without consent. Rambold, 54, is fighting the appeal. In a brief submitted to justices Thursday, Assistant Attorney General Tammy Plubell chastised Rambold for suggesting the victim should share in the blame. That assertion from Rambold was contained in arguments submitted by his attorneys. Victim Cherice Moralez killed herself while the case against Rambold was pending.

In [107]:
test_dataloader = DataLoader(tokenized_dataset["test"], batch_size=batch_size, shuffle=False)

In [116]:
for train_batch_idx, batch in enumerate(test_dataloader, start=1):
    train_inputs = batch['input_ids'].to(device)
    train_targets = batch['labels'].to(device)
    print(f'Sample text:')
    print(tokenizer.decode(train_inputs[0], skip_special_tokens=True))
    print()
    print(f'Expected:')
    print(tokenizer.decode(train_targets[0], skip_special_tokens=True))
    model.eval()
    with torch.no_grad():
        x = model.generate(train_inputs[0], tokenizer)
        print()
        print('Model answer:')
        print(tokenizer.decode(x, skip_special_tokens=True))
    break 

Sample text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's

In [113]:
loss_fn=nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
ts_epoch_losses = []
model.eval()
ts_epoch_loss = 0
with torch.no_grad():
    for ts_batch_idx, batch in enumerate(test_dataloader, start=1):
        ts_inputs = batch['input_ids'].to(device)
        ts_targets = batch['labels'].to(device)

        ts_preds = model(ts_inputs, ts_targets)

        ts_targets_shifted = ts_targets[:, 1:].contiguous().view(-1)
        ts_preds = ts_preds[:, :-1, :].contiguous().view(-1, ts_preds.size(-1))

        ts_batch_loss = loss_fn(ts_preds, ts_targets_shifted)
        ts_epoch_loss += ts_batch_loss.item()

ts_epoch_losses.append(ts_epoch_loss/len(test_dataloader))

In [114]:
ts_epoch_losses

[5.071946619283175]

## Fine tune a model from HuggingFace (2 pts)

Use a [guide](https://huggingface.co/docs/transformers/tasks/summarization) from HuggingFace🤗

In [117]:
!pip -q install accelerate -U
!pip -q install transformers[torch]

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [118]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

pretrained_model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [119]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [120]:
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
val_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64, shuffle=False)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="why_do_i_need_a_dir_for_that",
    eval_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.1,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=pretrained_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ····


wandb: ERROR API key must be 40 characters long, yours was 4
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

### Compare your models (2 pts)

Use rouge metric. See the example above. <br>


Feed your own texts to both model and see the result


Add a conclusion

In [ ]:
### METRICS EVALUATION ###

### My conclusion: